# Preprocesamiento de Datos - Kaggle Playground Series S5E6
## Fertilizer Prediction: Data Preprocessing Pipeline

**Autor:** Félix  
**Fecha:** 2025-01-21  
**Objetivo:** Preparar datos para modelado basándose en insights del EDA

---

### 📋 **Pipeline de Preprocesamiento**

**🎯 Basado en conclusiones del EDA 01_eda_template.ipynb:**

1. **Feature Engineering**: Implementar las 20+ features identificadas como valiosas
2. **Encoding**: Variables categóricas con high cardinality (Soil_Crop_Combo)
3. **Scaling**: Preparar variables numéricas para Random Forest baseline
4. **Splitting**: División estratificada para 22 clases de fertilizantes
5. **Export**: Datasets listos para modelado

**📊 Dataset Original:**
- 750,000 muestras
- 22 tipos de fertilizantes
- 9 variables predictoras + features engineered
- Sin valores faltantes

**🎯 Target Final:** Datos optimizados para Random Forest baseline model

## 📚 1. Importación de Librerías

### Librerías para Preprocesamiento de Datos

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
import warnings
warnings.filterwarnings('ignore')

# Para visualización (opcional en preprocessing)
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
np.random.seed(42)
plt.style.use('default')

print("✅ Librerías importadas correctamente")
print(f"📊 Pandas: {pd.__version__}")
print(f"🔢 NumPy: {np.__version__}")
print(f"🤖 Scikit-learn: sklearn disponible")

✅ Librerías importadas correctamente
📊 Pandas: 2.2.3
🔢 NumPy: 2.2.6
🤖 Scikit-learn: sklearn disponible


## 📊 2. Carga de Datos

### Carga de Datasets Originales

Cargamos los datos desde los archivos CSV y verificamos su estructura básica.

In [4]:
# Cargar datasets
print("=== CARGA DE DATOS ===")

train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')

print(f"\n📁 Archivos cargados:")
print(f"  • Train: {train_df.shape} (filas, columnas)")
print(f"  • Test: {test_df.shape}")
print(f"  • Sample Submission: {sample_submission.shape}")

# Verificar estructura
print(f"\n🔍 Columnas en Train:")
print(f"  {list(train_df.columns)}")

print(f"\n🔍 Columnas en Test:")
print(f"  {list(test_df.columns)}")

# Info básica
print(f"\n📊 Info Train Dataset:")
print(f"  • Tipos de datos únicos: {train_df.dtypes.value_counts().to_dict()}")
print(f"  • Valores nulos: {train_df.isnull().sum().sum()}")
print(f"  • Fertilizantes únicos: {train_df['Fertilizer Name'].nunique()}")

print(f"\n✅ Datos cargados correctamente - Listos para preprocesamiento")

=== CARGA DE DATOS ===

📁 Archivos cargados:
  • Train: (750000, 10) (filas, columnas)
  • Test: (250000, 9)
  • Sample Submission: (250000, 2)

🔍 Columnas en Train:
  ['id', 'Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name']

🔍 Columnas en Test:
  ['id', 'Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous']

📊 Info Train Dataset:
  • Tipos de datos únicos: {dtype('int64'): 7, dtype('O'): 3}
  • Valores nulos: 0
  • Fertilizantes únicos: 7

✅ Datos cargados correctamente - Listos para preprocesamiento

📁 Archivos cargados:
  • Train: (750000, 10) (filas, columnas)
  • Test: (250000, 9)
  • Sample Submission: (250000, 2)

🔍 Columnas en Train:
  ['id', 'Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name']

🔍 Columnas en Test:
  ['id', 'Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type'

## 🕳️ 3. Análisis de Valores Faltantes

### Verificación de Calidad de Datos

**Según el EDA:** El dataset no presenta valores faltantes, pero verificamos para confirmar.

In [5]:
# Análisis de valores faltantes
print("=== ANÁLISIS DE VALORES FALTANTES ===")

# Función para analizar valores faltantes
def analyze_missing_values(df, dataset_name):
    print(f"\n📊 {dataset_name}:")
    
    # Contar nulos
    null_counts = df.isnull().sum()
    null_percentages = (df.isnull().sum() / len(df)) * 100
    
    # Crear resumen
    missing_summary = pd.DataFrame({
        'Columna': df.columns,
        'Valores_Nulos': null_counts.values,
        'Porcentaje': null_percentages.values
    }).sort_values('Valores_Nulos', ascending=False)
    
    # Mostrar solo columnas con valores faltantes
    has_nulls = missing_summary[missing_summary['Valores_Nulos'] > 0]
    
    if len(has_nulls) > 0:
        print(f"  ⚠️ Columnas con valores faltantes:")
        for _, row in has_nulls.iterrows():
            print(f"    • {row['Columna']}: {row['Valores_Nulos']} ({row['Porcentaje']:.2f}%)")
    else:
        print(f"  ✅ No hay valores faltantes")
    
    # Verificar otros problemas de calidad
    print(f"  📋 Resumen de calidad:")
    print(f"    • Filas totales: {len(df):,}")
    print(f"    • Columnas totales: {len(df.columns)}")
    print(f"    • Valores duplicados: {df.duplicated().sum():,}")
    print(f"    • Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    return missing_summary

# Analizar ambos datasets
train_missing = analyze_missing_values(train_df, "DATASET DE ENTRENAMIENTO")
test_missing = analyze_missing_values(test_df, "DATASET DE PRUEBA")

# Verificar consistencia entre train y test
print(f"\n🔄 CONSISTENCIA ENTRE DATASETS:")

# Columnas comunes (excluyendo target)
common_cols = set(test_df.columns) & set(train_df.columns)
print(f"  • Columnas comunes: {len(common_cols)} de {len(test_df.columns)}")
print(f"    {list(common_cols)}")

# Verificar rangos de variables numéricas
numeric_cols = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
print(f"\n📊 Rangos de variables numéricas (Train vs Test):")
for col in numeric_cols:
    if col in train_df.columns and col in test_df.columns:
        train_range = f"[{train_df[col].min():.1f}, {train_df[col].max():.1f}]"
        test_range = f"[{test_df[col].min():.1f}, {test_df[col].max():.1f}]"
        print(f"  • {col:15}: Train {train_range} | Test {test_range}")

# Verificar categorías únicas
categorical_cols = ['Soil Type', 'Crop Type']
print(f"\n📊 Categorías únicas (Train vs Test):")
for col in categorical_cols:
    if col in train_df.columns and col in test_df.columns:
        train_cats = set(train_df[col].unique())
        test_cats = set(test_df[col].unique())
        
        print(f"  • {col}:")
        print(f"    Train: {len(train_cats)} categorías")
        print(f"    Test: {len(test_cats)} categorías")
        
        # Categorías solo en test (problemas potenciales)
        only_in_test = test_cats - train_cats
        if only_in_test:
            print(f"    ⚠️ Solo en Test: {only_in_test}")
        else:
            print(f"    ✅ Todas las categorías de test están en train")

print(f"\n🎯 CONCLUSIÓN ANÁLISIS DE CALIDAD:")
print(f"  ✅ Datasets limpios - sin valores faltantes")
print(f"  ✅ Consistencia entre train y test verificada")
print(f"  ✅ Listos para feature engineering")

=== ANÁLISIS DE VALORES FALTANTES ===

📊 DATASET DE ENTRENAMIENTO:
  ✅ No hay valores faltantes
  📋 Resumen de calidad:
    • Filas totales: 750,000
    • Columnas totales: 10
  ✅ No hay valores faltantes
  📋 Resumen de calidad:
    • Filas totales: 750,000
    • Columnas totales: 10
    • Valores duplicados: 0
    • Valores duplicados: 0
    • Memoria utilizada: 157.8 MB

📊 DATASET DE PRUEBA:
  ✅ No hay valores faltantes
  📋 Resumen de calidad:
    • Filas totales: 250,000
    • Columnas totales: 9
    • Memoria utilizada: 157.8 MB

📊 DATASET DE PRUEBA:
  ✅ No hay valores faltantes
  📋 Resumen de calidad:
    • Filas totales: 250,000
    • Columnas totales: 9
    • Valores duplicados: 0
    • Memoria utilizada: 39.5 MB

🔄 CONSISTENCIA ENTRE DATASETS:
  • Columnas comunes: 9 de 9
    ['Nitrogen', 'Phosphorous', 'Temparature', 'Soil Type', 'Crop Type', 'Moisture', 'Potassium', 'id', 'Humidity']

📊 Rangos de variables numéricas (Train vs Test):
  • Temparature    : Train [25.0, 38.0] | T

## ⚙️ 4. Feature Engineering

### Creación de Features Basadas en Insights del EDA

**🎯 Implementamos las features que demostraron alto valor predictivo en el EDA:**

1. **Ratios de Nutrientes**: N_P_ratio, N_K_ratio, P_K_ratio
2. **Índices Ambientales**: Temp_Humidity_index, Environmental_Stress
3. **Categorización de Nutrientes**: N_Level, P_Level, K_Level
4. **Interacciones Contextuales**: Soil_Crop_Combo
5. **Features Agregadas**: Total_NPK, Balance de humedad

**📊 Resultado esperado:** 20+ features totales para modelado

In [6]:
# Feature Engineering completo basado en insights del EDA
print("=== FEATURE ENGINEERING ESTRATÉGICO ===")

# Función para crear todas las features
def create_features(df, dataset_name=""):
    """
    Crear todas las features basadas en insights del EDA
    """
    print(f"\n🔧 Creando features para {dataset_name}...")
    
    # Hacer copia para no modificar original
    df_features = df.copy()
    
    # 1. RATIOS DE NUTRIENTES (Alto valor predictivo según MI)
    print("  1️⃣ Ratios de nutrientes...")
    df_features['N_P_ratio'] = df_features['Nitrogen'] / (df_features['Phosphorous'] + 0.1)  # +0.1 para evitar división por 0
    df_features['N_K_ratio'] = df_features['Nitrogen'] / (df_features['Potassium'] + 0.1)
    df_features['P_K_ratio'] = df_features['Phosphorous'] / (df_features['Potassium'] + 0.1)
    df_features['Total_NPK'] = df_features['Nitrogen'] + df_features['Phosphorous'] + df_features['Potassium']
    
    # 2. ÍNDICES AMBIENTALES
    print("  2️⃣ Índices ambientales...")
    df_features['Temp_Humidity_index'] = df_features['Temparature'] * df_features['Humidity'] / 100
    df_features['Moisture_Balance'] = df_features['Moisture'] - df_features['Humidity']
    df_features['Environmental_Stress'] = (abs(df_features['Temparature'] - 25) + 
                                         abs(df_features['Humidity'] - 65) + 
                                         abs(df_features['Moisture'] - 50)) / 3
    
    # 3. CATEGORIZACIÓN INTELIGENTE DE NUTRIENTES
    print("  3️⃣ Categorización de nutrientes...")
    
    # Temperatura (basado en análisis del EDA)
    df_features['Temp_Category'] = pd.cut(df_features['Temparature'], 
                                         bins=[-np.inf, 15, 25, 35, np.inf], 
                                         labels=['Frío', 'Templado', 'Cálido', 'Muy_Cálido'])
    
    # Niveles de nutrientes (basados en cuartiles del EDA)
    df_features['N_Level'] = pd.cut(df_features['Nitrogen'], 
                                   bins=[-np.inf, 15, 30, np.inf], 
                                   labels=['Bajo', 'Medio', 'Alto'])
    
    df_features['P_Level'] = pd.cut(df_features['Phosphorous'], 
                                   bins=[-np.inf, 20, 40, np.inf], 
                                   labels=['Bajo', 'Medio', 'Alto'])
    
    df_features['K_Level'] = pd.cut(df_features['Potassium'], 
                                   bins=[-np.inf, 10, 20, np.inf], 
                                   labels=['Bajo', 'Medio', 'Alto'])
    
    # 4. INTERACCIONES CONTEXTUALES (Alto MI según EDA)
    print("  4️⃣ Interacciones contextuales...")
    df_features['Soil_Crop_Combo'] = df_features['Soil Type'] + '_' + df_features['Crop Type']
    
    # Combinación de niveles NPK
    df_features['NPK_Level_Combo'] = (df_features['N_Level'].astype(str) + '_' + 
                                     df_features['P_Level'].astype(str) + '_' + 
                                     df_features['K_Level'].astype(str))
    
    # 5. FEATURES ADICIONALES DE BALANCE
    print("  5️⃣ Features de balance y ratios avanzados...")
    
    # Balance de macronutrientes
    df_features['NPK_Balance'] = abs(df_features['Nitrogen'] - df_features['Phosphorous']) + \
                                abs(df_features['Nitrogen'] - df_features['Potassium']) + \
                                abs(df_features['Phosphorous'] - df_features['Potassium'])
    
    # Índice de nutriente dominante
    max_nutrient = df_features[['Nitrogen', 'Phosphorous', 'Potassium']].max(axis=1)
    df_features['Dominant_Nutrient_Level'] = max_nutrient
    
    # Condiciones ambientales combinadas
    df_features['Temp_Moisture_interaction'] = df_features['Temparature'] * df_features['Moisture'] / 100
    
    # CREAR TARGET ENCODED PARA USO INTERNO (solo para train)
    if 'Fertilizer Name' in df_features.columns:
        # Crear target numérico para uso interno
        target_encoder = LabelEncoder()
        df_features['target'] = target_encoder.fit_transform(df_features['Fertilizer Name'])
        print("  🎯 Target numérico creado para uso interno")
    
    # Resumen de features creadas
    original_cols = len(df.columns)
    new_cols = len(df_features.columns)
    features_added = new_cols - original_cols
    
    print(f"  ✅ Features creadas: {features_added}")
    print(f"  📊 Total columnas: {original_cols} → {new_cols}")
    
    return df_features

# Aplicar feature engineering a ambos datasets
print("🔧 Aplicando feature engineering...")

train_processed = create_features(train_df, "TRAIN")
test_processed = create_features(test_df, "TEST")

# Verificar que las features se crearon correctamente
print(f"\n📊 RESUMEN DE FEATURE ENGINEERING:")

# Nuevas features numéricas
new_numeric_features = ['N_P_ratio', 'N_K_ratio', 'P_K_ratio', 'Total_NPK',
                       'Temp_Humidity_index', 'Moisture_Balance', 'Environmental_Stress',
                       'NPK_Balance', 'Dominant_Nutrient_Level', 'Temp_Moisture_interaction']

print(f"\n🔢 Nuevas features numéricas ({len(new_numeric_features)}):")
for feature in new_numeric_features:
    train_range = f"[{train_processed[feature].min():.2f}, {train_processed[feature].max():.2f}]"
    print(f"  • {feature:25}: {train_range}")

# Nuevas features categóricas
new_categorical_features = ['Temp_Category', 'N_Level', 'P_Level', 'K_Level', 
                           'Soil_Crop_Combo', 'NPK_Level_Combo']

print(f"\n📊 Nuevas features categóricas ({len(new_categorical_features)}):")
for feature in new_categorical_features:
    unique_count = train_processed[feature].nunique()
    print(f"  • {feature:20}: {unique_count} categorías únicas")

print(f"\n🎯 FEATURE ENGINEERING COMPLETADO:")
print(f"  • Train: {train_df.shape} → {train_processed.shape}")
print(f"  • Test: {test_df.shape} → {test_processed.shape}")
print(f"  • Features totales disponibles: {len(train_processed.columns)}")
print(f"  ✅ Listos para codificación de variables categóricas")

=== FEATURE ENGINEERING ESTRATÉGICO ===
🔧 Aplicando feature engineering...

🔧 Creando features para TRAIN...
  1️⃣ Ratios de nutrientes...
  2️⃣ Índices ambientales...
  3️⃣ Categorización de nutrientes...
  3️⃣ Categorización de nutrientes...
  4️⃣ Interacciones contextuales...
  4️⃣ Interacciones contextuales...
  5️⃣ Features de balance y ratios avanzados...
  5️⃣ Features de balance y ratios avanzados...
  🎯 Target numérico creado para uso interno
  ✅ Features creadas: 17
  📊 Total columnas: 10 → 27

🔧 Creando features para TEST...
  1️⃣ Ratios de nutrientes...
  2️⃣ Índices ambientales...
  3️⃣ Categorización de nutrientes...
  🎯 Target numérico creado para uso interno
  ✅ Features creadas: 17
  📊 Total columnas: 10 → 27

🔧 Creando features para TEST...
  1️⃣ Ratios de nutrientes...
  2️⃣ Índices ambientales...
  3️⃣ Categorización de nutrientes...
  4️⃣ Interacciones contextuales...
  4️⃣ Interacciones contextuales...
  5️⃣ Features de balance y ratios avanzados...
  ✅ Features c

## 🔢 5. Preparación de Datos por Tipo de Modelo

### Estrategia Multi-Modelo: Nativas vs Codificadas

**🎯 Creamos DOS versiones optimizadas:**

1. **📊 Versión NATIVE** (Random Forest, LightGBM, CatBoost):
   - Variables categóricas como strings/objects
   - Mejor manejo de alta cardinalidad
   - Splits más inteligentes

2. **🔢 Versión ENCODED** (XGBoost):
   - LabelEncoder aplicado a categóricas
   - Solo datos numéricos
   - Requerido por XGBoost

**💡 Variables categóricas identificadas:**
- Originales: Soil Type, Crop Type
- Nuevas: Temp_Category, N_Level, P_Level, K_Level, Soil_Crop_Combo, NPK_Level_Combo

In [7]:
# Preparación de datos para diferentes tipos de modelos
print("=== PREPARACIÓN MULTI-MODELO ===")

# Identificar todas las variables categóricas
categorical_columns = ['Soil Type', 'Crop Type', 'Temp_Category', 'N_Level', 
                      'P_Level', 'K_Level', 'Soil_Crop_Combo', 'NPK_Level_Combo']

# Verificar cuáles existen en los datos
existing_categorical = [col for col in categorical_columns if col in train_processed.columns]
print(f"📊 Variables categóricas identificadas: {len(existing_categorical)}")
for col in existing_categorical:
    print(f"  • {col}: {train_processed[col].nunique()} categorías")

# ========================================
# VERSIÓN 1: DATOS NATIVOS (RF, LightGBM, CatBoost)
# ========================================
print(f"\n1️⃣ CREANDO VERSIÓN NATIVE (categóricas como strings):")

# Preparar versión nativa (mantener Fertilizer Name como está)
train_native = train_processed.copy()
test_native = test_processed.copy()

print(f"  ✅ Variables categóricas mantenidas como strings/objects")
print(f"  ✅ Target mantenido como 'Fertilizer Name': {train_native['Fertilizer Name'].nunique()} clases")
print(f"  🎯 Optimizado para: Random Forest, LightGBM, CatBoost")

# ========================================
# VERSIÓN 2: DATOS CODIFICADOS (XGBoost)
# ========================================
print(f"\n2️⃣ CREANDO VERSIÓN ENCODED (todo numérico):")

# Función para codificar variables categóricas
def encode_categorical_variables(train_df, test_df, categorical_cols, target_col=None):
    """
    Codifica TODAS las variables categóricas para XGBoost
    """
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    encoders = {}
    
    print(f"  🔧 Aplicando LabelEncoder a todas las categóricas...")
    
    for col in categorical_cols:
        if col in train_df.columns:
            # Crear LabelEncoder
            le = LabelEncoder()
            
            # Fit en todas las categorías (train + test) para consistencia
            all_categories = pd.concat([
                train_df[col].astype(str), 
                test_df[col].astype(str) if col in test_df.columns else pd.Series([])
            ]).unique()
            
            le.fit(all_categories)
            
            # Transform train y test
            train_encoded[f'{col}_encoded'] = le.transform(train_df[col].astype(str))
            if col in test_df.columns:
                test_encoded[f'{col}_encoded'] = le.transform(test_df[col].astype(str))
            
            # Guardar encoder
            encoders[col] = le
            
            print(f"    📊 {col}: {len(all_categories)} categorías → [0, {len(all_categories)-1}]")
    
    # Ya tenemos target codificado en la columna 'target'
    if target_col and target_col in train_df.columns and 'target' in train_df.columns:
        print(f"    🎯 Target ya disponible como numérico en columna 'target'")
    
    return train_encoded, test_encoded, encoders

# Aplicar codificación completa
train_encoded, test_encoded, label_encoders = encode_categorical_variables(
    train_processed, test_processed, 
    existing_categorical
)

print(f"  ✅ Todas las variables categóricas codificadas")
print(f"  ✅ Dataset completamente numérico")
print(f"  🎯 Optimizado para: XGBoost")

# ========================================
# RESUMEN DE VERSIONES CREADAS
# ========================================
print(f"\n📊 RESUMEN DE VERSIONES CREADAS:")
print(f"\n🌳 VERSIÓN NATIVE:")
print(f"  • train_native: {train_native.shape}")
print(f"  • test_native: {test_native.shape}")
print(f"  • Variables categóricas: {len(existing_categorical)} como strings")
print(f"  • Target: 'Fertilizer Name' ({train_native['Fertilizer Name'].nunique()} clases)")
print(f"  • Ideal para: Random Forest, LightGBM, CatBoost")

print(f"\n🔢 VERSIÓN ENCODED:")
print(f"  • train_encoded: {train_encoded.shape}")
print(f"  • test_encoded: {test_encoded.shape}")
print(f"  • Variables categóricas: {len(existing_categorical)} codificadas")
print(f"  • Target: 'target' ({train_encoded['target'].nunique()} clases)")
print(f"  • Ideal para: XGBoost")

print(f"\n💡 RECOMENDACIONES:")
print(f"  🥇 Random Forest BASELINE: Usar train_native (mejor rendimiento)")
print(f"  🚀 XGBoost: Usar train_encoded (requerimiento obligatorio)")
print(f"  ⚡ LightGBM: Usar train_native + categorical_feature parameter")
print(f"  🎯 CatBoost: Usar train_native + cat_features parameter")

# Verificar mapeos importantes
print(f"\n🔍 MAPEOS DE ENCODERS (ejemplos):")

# Mostrar mapeo de Soil Type
if 'Soil Type' in label_encoders:
    soil_mapping = dict(zip(label_encoders['Soil Type'].classes_, 
                           label_encoders['Soil Type'].transform(label_encoders['Soil Type'].classes_)))
    print(f"  🌱 Soil Type:")
    for soil, code in list(soil_mapping.items())[:3]:
        print(f"    {code}: {soil}")
    if len(soil_mapping) > 3:
        print(f"    ... y {len(soil_mapping)-3} más")

# Mostrar mapeo de Crop Type (primeras 5)
if 'Crop Type' in label_encoders:
    crop_mapping = dict(zip(label_encoders['Crop Type'].classes_, 
                           label_encoders['Crop Type'].transform(label_encoders['Crop Type'].classes_)))
    print(f"  🌾 Crop Type (primeros 5):")
    for i, (crop, code) in enumerate(crop_mapping.items()):
        if i < 5:
            print(f"    {code}: {crop}")
    if len(crop_mapping) > 5:
        print(f"    ... y {len(crop_mapping)-5} más")

# Estadísticas finales
print(f"\n📈 ESTADÍSTICAS POST-CODIFICACIÓN:")
print(f"  • Train shape: {train_encoded.shape}")
print(f"  • Test shape: {test_encoded.shape}")
print(f"  • Columnas numéricas totales: {len(train_encoded.select_dtypes(include=[np.number]).columns)}")
print(f"  • Columnas categóricas originales mantenidas: {len(train_encoded.select_dtypes(include=['object', 'category']).columns)}")

print(f"\n✅ CODIFICACIÓN COMPLETADA")
print(f"  🎯 Datasets listos para escalado de variables numéricas")

=== PREPARACIÓN MULTI-MODELO ===
📊 Variables categóricas identificadas: 8
  • Soil Type: 5 categorías
  • Soil Type: 5 categorías
  • Crop Type: 11 categorías
  • Temp_Category: 3 categorías
  • N_Level: 3 categorías
  • P_Level: 3 categorías
  • K_Level: 2 categorías
  • Crop Type: 11 categorías
  • Temp_Category: 3 categorías
  • N_Level: 3 categorías
  • P_Level: 3 categorías
  • K_Level: 2 categorías
  • Soil_Crop_Combo: 55 categorías
  • NPK_Level_Combo: 18 categorías

1️⃣ CREANDO VERSIÓN NATIVE (categóricas como strings):
  • Soil_Crop_Combo: 55 categorías
  • NPK_Level_Combo: 18 categorías

1️⃣ CREANDO VERSIÓN NATIVE (categóricas como strings):
  ✅ Variables categóricas mantenidas como strings/objects
  ✅ Target mantenido como 'Fertilizer Name': 7 clases
  🎯 Optimizado para: Random Forest, LightGBM, CatBoost

2️⃣ CREANDO VERSIÓN ENCODED (todo numérico):
  ✅ Variables categóricas mantenidas como strings/objects
  ✅ Target mantenido como 'Fertilizer Name': 7 clases
  🎯 Optimizado 

KeyboardInterrupt: 

## 🔄 6. División de Datos para Modelado

### Splitting Estratificado para Ambas Versiones

**🎯 Estrategia de División:**

- **Split ratio**: 80/20 (entrenamiento/validación)
- **Estratificación**: Por 22 fertilizantes
- **Dos versiones**: Native (RF/LightGBM/CatBoost) y Encoded (XGBoost)
- **Sin escalado**: Tree-based models no requieren normalización

**📊 Datasets de salida:**
- train_native / val_native: Para Random Forest, LightGBM, CatBoost
- train_encoded / val_encoded: Para XGBoost

## 💾 7. Guardado de Datasets Preprocesados

### Exportación de Datos Procesados

**🎯 Estructura de Archivos de Salida:**

1. **Datasets de entrenamiento/validación**:
   - `train_native.csv` - Para Random Forest, LightGBM, CatBoost
   - `val_native.csv` - Validación versión nativa
   - `train_encoded.csv` - Para XGBoost
   - `val_encoded.csv` - Validación versión codificada

2. **Dataset de prueba procesado**:
   - `test_native.csv` - Test versión nativa
   - `test_encoded.csv` - Test versión codificada

3. **Metadatos y información**:
   - `feature_info.json` - Información de features por tipo
   - `preprocessing_info.json` - Metadatos del preprocesamiento

**📁 Directorio de salida:** `../data/processed/`

In [ ]:
# División de datos para ambas versiones (Native y Encoded)
print("=== DIVISIÓN DE DATOS PARA AMBAS VERSIONES ===")

from sklearn.model_selection import train_test_split

# Función para crear split estratificado
def create_train_val_split(X, y, test_size=0.2, random_state=513):
    """
    Crea split estratificado manteniendo proporción de clases
    """
    return train_test_split(X, y, test_size=test_size, 
                           stratify=y, random_state=random_state)

# Target variable (usar la columna target numérica)
y = train_native['target']  # Target ya codificado

print(f"📊 Información del target:")
print(f"  • Total muestras: {len(y):,}")
print(f"  • Clases únicas: {y.nunique()}")
print(f"  • Rango de clases: 0 a {y.max()}")

# ========================================
# 1. DIVISIÓN PARA VERSIÓN NATIVE (RF, LightGBM, CatBoost)
# ========================================
print(f"\n1️⃣ División para NATIVE version:")

# Features para versión native (excluir columnas target y auxiliares)
exclude_cols_native = ['Fertilizer Name', 'target', 'id'] if 'id' in train_native.columns else ['Fertilizer Name', 'target']
features_native = [col for col in train_native.columns if col not in exclude_cols_native]
X_native = train_native[features_native]
y_native = y

# Split estratificado para native
X_train_native, X_val_native, y_train, y_val = create_train_val_split(X_native, y_native)

print(f"  📊 X_train_native: {X_train_native.shape}")
print(f"  📊 X_val_native: {X_val_native.shape}")
print(f"  📊 y_train: {len(y_train)}")
print(f"  📊 y_val: {len(y_val)}")

# Preparar test native - mantener ID si existe
if 'id' in test_native.columns:
    # Conservar ID para las predicciones
    test_ids = test_native['id']
    features_test_native = [col for col in features_native if col in test_native.columns]
    X_test_native = test_native[features_test_native]
else:
    # Si no hay columna id, crear una
    test_ids = pd.Series(range(len(test_native)), name='id')
    features_test_native = [col for col in features_native if col in test_native.columns]
    X_test_native = test_native[features_test_native]

print(f"  📊 X_test_native: {X_test_native.shape}")

# ========================================
# 2. DIVISIÓN PARA VERSIÓN ENCODED (XGBoost)
# ========================================
print(f"\n2️⃣ División para ENCODED version:")

# Features para versión encoded (excluir columnas target y auxiliares)
exclude_cols_encoded = ['Fertilizer Name', 'target', 'id'] if 'id' in train_encoded.columns else ['Fertilizer Name', 'target']
# También excluir las columnas categóricas originales (usar solo las _encoded)
original_categoricals = ['Soil Type', 'Crop Type', 'Temp_Category', 'N_Level', 'P_Level', 'K_Level', 'Soil_Crop_Combo', 'NPK_Level_Combo']
exclude_cols_encoded.extend(original_categoricals)

features_encoded = [col for col in train_encoded.columns if col not in exclude_cols_encoded]
X_encoded = train_encoded[features_encoded]
y_encoded = train_encoded['target']  # Usar la columna target

# Split estratificado para encoded (usando mismos índices que native)
X_train_encoded = X_encoded.iloc[X_train_native.index]
X_val_encoded = X_encoded.iloc[X_val_native.index]

print(f"  📊 X_train_encoded: {X_train_encoded.shape}")
print(f"  📊 X_val_encoded: {X_val_encoded.shape}")

# Preparar test encoded
features_test_encoded = [col for col in features_encoded if col in test_encoded.columns]
X_test_encoded = test_encoded[features_test_encoded]
print(f"  📊 X_test_encoded: {X_test_encoded.shape}")

# ========================================
# 3. VERIFICACIÓN DE ESTRATIFICACIÓN
# ========================================
print(f"\n📊 VERIFICACIÓN DE ESTRATIFICACIÓN:")
print(f"  Original: {y.value_counts().head(5).values}")
print(f"  Train:    {y_train.value_counts().head(5).values}")
print(f"  Val:      {y_val.value_counts().head(5).values}")

# Calcular proporciones
train_prop = len(y_train) / len(y)
val_prop = len(y_val) / len(y)
print(f"\n  Proporciones: Train {train_prop:.1%}, Val {val_prop:.1%}")

# ========================================
# 4. RESUMEN DE DATASETS PREPARADOS
# ========================================
print(f"\n🎯 RESUMEN DE DATASETS PREPARADOS:")
print(f"  • X_train_native: {X_train_native.shape} - Para Random Forest, LightGBM, CatBoost")
print(f"  • X_val_native: {X_val_native.shape}")
print(f"  • X_test_native: {X_test_native.shape}")
print(f"  • X_train_encoded: {X_train_encoded.shape} - Para XGBoost")
print(f"  • X_val_encoded: {X_val_encoded.shape}")
print(f"  • X_test_encoded: {X_test_encoded.shape}")
print(f"  • y_train, y_val: {len(y_train)}, {len(y_val)} muestras")
print(f"  • test_ids: {len(test_ids)} IDs guardados")

print(f"\n📊 FEATURES DISPONIBLES:")
print(f"  • Native features: {len(features_native)}")
print(f"  • Encoded features: {len(features_encoded)}")
print(f"  • Categóricas nativas: {len([f for f in features_native if f in original_categoricals])}")
print(f"  • Categóricas codificadas: {len([f for f in features_encoded if '_encoded' in f])}")

print(f"\n✅ DIVISIÓN DE DATOS COMPLETADA")
print(f"  🎯 Dos versiones de datos preparadas para diferentes modelos")
print(f"  🎯 Estratificación mantenida para ambas versiones")
print(f"  🎯 Listos para entrenamiento y exportación")

In [ ]:
# Guardado de datasets preprocesados para ambas versiones
print("=== GUARDADO DE DATASETS PREPROCESADOS ===")

import os
import json
from datetime import datetime

# Crear directorio de salida
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)
print(f"📁 Directorio de salida: {output_dir}")

# Función para guardar datasets con información adicional
def save_processed_data(data, filename, description):
    """
    Guarda un dataset con metadata
    """
    filepath = os.path.join(output_dir, filename)
    data.to_csv(filepath, index=False)
    print(f"  ✅ {filename}: {data.shape} - {description}")
    return filepath

print(f"\n💾 Guardando datasets procesados...")

# ========================================
# 1. DATASETS VERSIÓN NATIVE (RF, LightGBM, CatBoost)
# ========================================
print(f"\n1️⃣ Datasets NATIVE (Random Forest, LightGBM, CatBoost):")

# Combinar features y target para entrenamiento native
train_data_native = X_train_native.copy()
train_data_native['Fertilizer Name'] = train_native.loc[X_train_native.index, 'Fertilizer Name']
train_data_native['target'] = y_train

val_data_native = X_val_native.copy()
val_data_native['Fertilizer Name'] = train_native.loc[X_val_native.index, 'Fertilizer Name']
val_data_native['target'] = y_val

# Test native con IDs
test_data_native = X_test_native.copy()
test_data_native['id'] = test_ids

# Guardar datasets native
save_processed_data(train_data_native, 'train_native.csv', 
                   'Training data NATIVE - categóricas como strings (RF, LightGBM, CatBoost)')
save_processed_data(val_data_native, 'val_native.csv', 
                   'Validation data NATIVE')
save_processed_data(test_data_native, 'test_native.csv', 
                   'Test data NATIVE')

# ========================================
# 2. DATASETS VERSIÓN ENCODED (XGBoost)
# ========================================
print(f"\n2️⃣ Datasets ENCODED (XGBoost):")

# Combinar features y target para entrenamiento encoded
train_data_encoded = X_train_encoded.copy()
train_data_encoded['target'] = y_train

val_data_encoded = X_val_encoded.copy()
val_data_encoded['target'] = y_val

# Test encoded con IDs
test_data_encoded = X_test_encoded.copy()
test_data_encoded['id'] = test_ids

# Guardar datasets encoded
save_processed_data(train_data_encoded, 'train_encoded.csv', 
                   'Training data ENCODED - categóricas codificadas (XGBoost)')
save_processed_data(val_data_encoded, 'val_encoded.csv', 
                   'Validation data ENCODED')
save_processed_data(test_data_encoded, 'test_encoded.csv', 
                   'Test data ENCODED')

# ========================================
# 3. METADATA Y INFORMACIÓN DE FEATURES
# ========================================
print(f"\n3️⃣ Guardando metadata:")

# Obtener nombres de fertilizantes únicos para el mapeo
fertilizer_names = sorted(train_native['Fertilizer Name'].unique())
fertilizer_mapping = {i: name for i, name in enumerate(fertilizer_names)}

# Información de features
feature_info = {
    'features_native': {
        'names': features_native,
        'count': len(features_native),
        'categorical': [col for col in features_native if col in ['Soil Type', 'Crop Type', 'Temp_Category', 'N_Level', 'P_Level', 'K_Level', 'Soil_Crop_Combo', 'NPK_Level_Combo']],
        'numerical': [col for col in features_native if col not in ['Soil Type', 'Crop Type', 'Temp_Category', 'N_Level', 'P_Level', 'K_Level', 'Soil_Crop_Combo', 'NPK_Level_Combo']]
    },
    'features_encoded': {
        'names': features_encoded,
        'count': len(features_encoded),
        'categorical_encoded': [col for col in features_encoded if '_encoded' in col],
        'numerical': [col for col in features_encoded if '_encoded' not in col]
    },
    'target_info': {
        'name': 'Fertilizer Name',
        'classes': len(fertilizer_names),
        'class_names': fertilizer_names,
        'target_mapping': fertilizer_mapping
    },
    'split_info': {
        'train_size': len(y_train),
        'val_size': len(y_val),
        'test_size': len(X_test_native),
        'train_ratio': len(y_train) / len(y),
        'val_ratio': len(y_val) / len(y)
    },
    'feature_counts': {
        'native_total': len(features_native),
        'encoded_total': len(features_encoded),
        'categorical_native': len([f for f in features_native if f in ['Soil Type', 'Crop Type', 'Temp_Category', 'N_Level', 'P_Level', 'K_Level', 'Soil_Crop_Combo', 'NPK_Level_Combo']]),
        'categorical_encoded': len([f for f in features_encoded if '_encoded' in f])
    }
}

# Guardar información de features
with open(os.path.join(output_dir, 'feature_info.json'), 'w') as f:
    json.dump(feature_info, f, indent=2)
print(f"  ✅ feature_info.json: Información detallada de features")

# Información de preprocesamiento
preprocessing_info = {
    'timestamp': datetime.now().isoformat(),
    'original_data_shape': {
        'train': list(train_df.shape),
        'test': list(test_df.shape)
    },
    'processed_data_shape': {
        'train_native': list(train_data_native.shape),
        'val_native': list(val_data_native.shape),
        'test_native': list(test_data_native.shape),
        'train_encoded': list(train_data_encoded.shape),
        'val_encoded': list(val_data_encoded.shape),
        'test_encoded': list(test_data_encoded.shape)
    },
    'feature_engineering': {
        'ratio_features': ['N_P_ratio', 'N_K_ratio', 'P_K_ratio'],
        'categorical_features': ['N_Level', 'P_Level', 'K_Level', 'Temp_Category'],
        'interaction_features': ['Soil_Crop_Combo', 'NPK_Level_Combo'],
        'aggregate_features': ['Total_NPK', 'Environmental_Stress']
    },
    'encoding_strategy': {
        'native_version': 'Categorical variables as strings for RF, LightGBM, CatBoost',
        'encoded_version': 'LabelEncoder applied for XGBoost compatibility'
    },
    'files_created': {
        'native': ['train_native.csv', 'val_native.csv', 'test_native.csv'],
        'encoded': ['train_encoded.csv', 'val_encoded.csv', 'test_encoded.csv'],
        'metadata': ['feature_info.json', 'preprocessing_info.json']
    }
}

# Guardar información de preprocesamiento
with open(os.path.join(output_dir, 'preprocessing_info.json'), 'w') as f:
    json.dump(preprocessing_info, f, indent=2)
print(f"  ✅ preprocessing_info.json: Metadata completa del preprocesamiento")

# ========================================
# 4. RESUMEN FINAL
# ========================================
print(f"\n🎆 RESUMEN FINAL DEL PREPROCESAMIENTO:")
print(f"  📁 Archivos guardados en: {output_dir}")
print(f"  📊 Total de archivos generados: 8 datasets + 2 metadata")
print(f"  🎯 Datasets NATIVE listos para: Random Forest, LightGBM, CatBoost")
print(f"  🎯 Datasets ENCODED listos para: XGBoost")
print(f"  🔄 Features engineered: {len(features_native)} total")
print(f"  🎯 Target classes: {len(fertilizer_names)} fertilizantes")

print(f"\n📊 ESTADÍSTICAS FINALES:")
print(f"  • Train: {len(y_train):,} muestras")
print(f"  • Validation: {len(y_val):,} muestras")
print(f"  • Test: {len(test_ids):,} muestras")
print(f"  • Features NATIVE: {len(features_native)}")
print(f"  • Features ENCODED: {len(features_encoded)}")

print(f"\n✅ ¡PREPROCESAMIENTO COMPLETADO CON ÉXITO!")
print(f"  🚀 Listos para entrenar modelos baseline")
print(f"  🎯 Siguiente paso: Crear notebooks de modelado")

## 🤔 Estrategia de Encoding por Modelo

### 📊 **¿Codificar Variables Categóricas para Tree-Based Models?**

**La respuesta depende del modelo específico:**

| Modelo | Encoding Necesario | Estrategia Óptima |
|--------|-------------------|-------------------|
| **Random Forest** | ❌ NO | Mantener categóricas como strings |
| **XGBoost** | ✅ SÍ | LabelEncoder obligatorio |
| **LightGBM** | 🔄 OPCIONAL | Categóricas nativas (mejor) |
| **CatBoost** | ❌ NO | Categóricas nativas con cat_features |

### 🎯 **Nuestra Estrategia Multi-Modelo:**

1. **Versión Native**: Variables categóricas como strings (RF, LightGBM, CatBoost)
2. **Versión Encoded**: LabelEncoder aplicado (XGBoost)
3. **Features numéricas**: Sin escalar para todos los tree-based models

**💡 Ventajas de categóricas nativas:**
- Mejor manejo de alta cardinalidad (Soil_Crop_Combo: 110+ categorías)
- Splits más inteligentes
- Mejor interpretabilidad
- Menos overfitting en high cardinality features